In [6]:
import os
import requests
from dotenv import load_dotenv

import pandas as pd
load_dotenv()

True

In [22]:
import plotly.graph_objects as go
from price_data_loader.load_price_data_alpha_vantage import LoadExampleData

loader = LoadExampleData()
loader.load()

loader.balance_sheet_annual

price_df = loader.daily_time_series
dividend_df = loader.dividends
symbol = price_df.symbol.unique().item()

print(price_df)
print(dividend_df)

            date symbol    open      high       low   close    volume
0     1999-11-01    IBM   98.50   98.8100   96.3700   96.75   9551800
1     1999-11-02    IBM   96.75   96.8100   93.6900   94.81  11105400
2     1999-11-03    IBM   95.87   95.9400   93.5000   94.37  10369100
3     1999-11-04    IBM   94.44   94.4400   90.0000   91.56  16697600
4     1999-11-05    IBM   92.75   92.9400   90.1900   90.25  13737600
...          ...    ...     ...       ...       ...     ...       ...
6329  2024-12-27    IBM  223.14  224.4200  221.4054  222.78   1810760
6330  2024-12-30    IBM  220.54  221.5942  217.6523  220.25   2095565
6331  2024-12-31    IBM  220.72  221.0493  218.4400  219.83   2270512
6332  2025-01-02    IBM  221.82  222.4900  217.6000  219.94   2579498
6333  2025-01-03    IBM  220.55  223.6600  220.5500  222.65   3873578

[6334 rows x 7 columns]
    symbol  amount ex_dividend_date declaration_date record_date payment_date
0      IBM    0.22       1999-02-08              NaN     

In [26]:
price_df.loc[price_df['date'] == '2024-12-27', 'close']

6329    222.78
Name: close, dtype: float64

In [32]:
set(dividend_df.ex_dividend_date.to_list()).difference(set(price_df.date.to_list()))

{'1999-02-08', '1999-05-06', '1999-08-06'}

In [31]:
dividend_df.shape[0]

104

In [20]:
[price_df.loc[price_df['date'] == date, 'close'].values[0] for date in dividend_df["ex_dividend_date"]]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:

fig = go.Figure(data=[go.Candlestick(x=price_df['date'],
                                     open=price_df['open'],
                                     high=price_df['high'],
                                     low=price_df['low'],
                                     close=price_df['close'])])

fig.add_trace(go.Scatter(
    x=dividend_df["ex_dividend_date"],
    y=[price_df.loc[price_df['date'] == date, 'close'].values[0] for date in dividend_df["ex_dividend_date"]],
    mode="markers+text",
    marker=dict(size=10, color="red"),
    text=[f"Div: {d}" for d in dividend_df["amount"]],
    textposition="top center",
    name="Dividends"
))


fig.update_layout(title=dict(text=f'All daily price data for {symbol}'),
                  yaxis=dict(title=dict(text=f'{symbol} price')),
                  xaxis_rangeslider_visible=False)
fig.show()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
df

,date,symbol,open,high,low,close,volume
0,1999-11-01,IBM,98.50,98.8100,96.3700,96.75,9551800
1,1999-11-02,IBM,96.75,96.8100,93.6900,94.81,11105400
2,1999-11-03,IBM,95.87,95.9400,93.5000,94.37,10369100
3,1999-11-04,IBM,94.44,94.4400,90.0000,91.56,16697600
4,1999-11-05,IBM,92.75,92.9400,90.1900,90.25,13737600
...,...,...,...,...,...,...,...
6329,2024-12-27,IBM,223.14,224.4200,221.4054,222.78,1810760
6330,2024-12-30,IBM,220.54,221.5942,217.6523,220.25,2095565
6331,2024-12-31,IBM,220.72,221.0493,218.4400,219.83,2270512
6332,2025-01-02,IBM,221.82,222.4900,217.6000,219.94,2579498


In [16]:
import pandas as pd
import plotly.graph_objects as go

# Example candlestick data
data = {
    "Date": pd.date_range(start="2023-01-01", periods=100),
    "Open": [100 + i * 0.5 for i in range(100)],
    "High": [105 + i * 0.5 for i in range(100)],
    "Low": [95 + i * 0.5 for i in range(100)],
    "Close": [102 + i * 0.5 for i in range(100)],
}
df = pd.DataFrame(data)

# Quarterly dividends data
dividends = pd.DataFrame({
    "Date": ["2023-03-31", "2023-06-30", "2023-09-30", "2023-12-31"],
    "Dividend": [1.0, 1.2, 1.1, 1.3],
})
dividends["Date"] = pd.to_datetime(dividends["Date"])

# Find the closest matching candlestick date for each dividend
dividends["Closest_Date"] = dividends["Date"].apply(
    lambda d: df.loc[(df['Date'] - d).abs().idxmin(), 'Date']
)

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=df['Date'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Candlestick'
)])

# Add dividends as scatter points
fig.add_trace(go.Scatter(
    x=dividends["Closest_Date"],
    y=[df.loc[df['Date'] == date, 'Close'].values[0] for date in dividends["Closest_Date"]],
    mode="markers+text",
    marker=dict(size=10, color="red"),
    text=[f"Div: {d}" for d in dividends["Dividend"]],
    textposition="top center",
    name="Dividends"
))

# Update layout
fig.update_layout(
    title="Candlestick Chart with Quarterly Dividends",
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
)

fig.show()
